# Transfer Learning with PyTorch using Azure Machine Learning

In [2]:
import os
import azureml
import shutil
import urllib.request
from pathlib import Path
import urllib3
import zipfile

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment, Run, Environment, ScriptRunConfig

from azureml.core.compute import ComputeTarget, AmlCompute, AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails

from azureml.core.webservice import Webservice, AksWebservice, AciWebservice
from azureml.core.dataset import Dataset
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.conda_dependencies import CondaDependencies 

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

ModuleNotFoundError: No module named 'azureml.core'

## 1 - Connect to the AML Workspace Environment

In [ ]:
# Connect to workspace
ws = Workspace.from_config()
print("Workspace:",ws.name)

# Connect to compute for training
gpu_instance_name = "weak-supervisor-gpu"

# Verify that cluster does not exist already
try:
    gpu_instance = ComputeTarget(workspace=ws, name=gpu_instance_name)
    print('Found existing gpu instance, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12s_v3',
max_nodes=1)
    gpu_instance = ComputeTarget.create(ws, gpu_instance_name, compute_config)

gpu_instance.wait_for_completion(show_output=True)

print("Compute Target:",gpu_instance.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:",ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='Weak-Supervision-fsdl')
print("Experiment:",exp.name)

## 2 - Data

In [ ]:
# Get data by uploading dataset to Azure
toxic_comments_ds = Dataset.get_by_name(ws, name='toxic_comments')
df = toxic_comments_ds.to_pandas_dataframe()

In [ ]:
df_toxic = df[df[LABEL_COLUMNS].sum(axis=1) > 0]
df_clean = df[df[LABEL_COLUMNS].sum(axis=1) == 0]

train_df = pd.concat([
  df_toxic.sample(2000, random_state=42),
  df_clean.sample(2000, random_state=42)
])

train_df['label'] = np.where(train_df[LABEL_COLUMNS].sum(axis=1) == 0, 0, 1)
train_df = train_df.drop(LABEL_COLUMNS, axis=1)
train_df = train_df.rename(columns={"comment_text": "text"})

In [ ]:
fine_tune_df = train_df.groupby('label').apply(lambda s: s.sample(500, random_state=123)).reset_index(level=0, drop=True)
train_df.drop(fine_tune_df.index, inplace=True)

train_df, test_df = train_test_split(train_df, test_size=0.25)
val_df = test_df.sample(frac=0.1)
test_df.drop(val_df.index, inplace=True)

print('\t Fine-Tune Set:', len(fine_tune_df), 'Valid:', len(val_df), '\t', 'Train:', len(train_df), '\t Test:', len(test_df),)

## 3 - Train the model

### 3.1 Create a training script

In [ ]:
%%writefile 'trainingscripts/train.py'

from __future__ import print_function, division

import pandas as pd
import numpy as np
import os
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning, QuantizationAwareTraining, ModelPruning
from pytorch_lightning.loggers import TensorBoardLogger
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)
# lightning plus wandb
import wandb
from pytorch_lightning.loggers import WandbLogger

import gc

### Add References
import argparse
from azureml.core import Run

### Add run context for AML
run = Run.get_context()

### Parse incoming parameters
parser = argparse.ArgumentParser()
parser.add_argument("--data-folder", type=str, dest="data_folder", help="data folder mounting point", default="")
parser.add_argument("--num-epochs", type=int, dest="num_epochs", help="Number of epochs", default="")
parser.add_argument("--max_token_count", type=int, dest="max_token_count", help="max_token_count", default="")
parser.add_argument("--batch_size", type=int, dest="batch_size", help="batch_size", default="")

args = parser.parse_args()
data_path = args.data_folder
num_epochs = args.num_epochs

class ToxicCommentsDataset(Dataset):

  def __init__(
    self, 
    data: pd.DataFrame,
    tokenizer: DistilBertForSequenceClassification,
    max_token_len: int = 128
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    comment_text = data_row.text
    labels = data_row['label']

    encoding = self.tokenizer.encode_plus(
      comment_text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      comment_text=comment_text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels=labels
    )


train_dataset = ToxicCommentsDataset(
  fine_tune_df,
  tokenizer=tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

class ToxicCommentDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=4, max_token_len=128):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    self.train_dataset = ToxicCommentsDataset(
      self.train_df,
      self.tokenizer,
      self.max_token_len
    )

    self.test_dataset = ToxicCommentsDataset(
      self.test_df,
      self.tokenizer,
      self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

data_module = ToxicCommentDataModule(
  fine_tune_df,
  val_df,
  tokenizer=tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)


class ToxicCommentTagger(pl.LightningModule):

  def __init__(self, num_classes: int, input_dims=(BATCH_SIZE, MAX_TOKEN_COUNT), attn_mask=(BATCH_SIZE, MAX_TOKEN_COUNT), n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.backbone = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    # self.backbone = DistilBertForSequenceClassification.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.backbone.config.hidden_size, num_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.CrossEntropyLoss()

    # log hyperparameters
    self.save_hyperparameters()

    # compute the accuracy -- no need to roll your own!
    self.train_acc = pl.metrics.Accuracy()
    self.val_acc = pl.metrics.Accuracy()
    self.test_acc = pl.metrics.Accuracy()
    self.train_f1 = pl.metrics.F1(num_classes=num_classes)
    self.val_f1 = pl.metrics.F1(num_classes=num_classes)
    self.test_f1 = pl.metrics.F1(num_classes=num_classes)

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.backbone(input_ids, attention_mask=attention_mask)
    # output = torch.argmax(output.logits, 1)
    # print(output.pooler_output)
    output = self.classifier(output.pooler_output)
    # print(output)
    output = torch.sigmoid(output)
    # print(output)
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels) # calls self.forward
    preds = torch.argmax(outputs, 1) # take the highest predicted number

    self.train_acc(preds, labels)
    self.train_f1(preds, labels)
    self.log('train/loss_epoch', loss, on_step=False, on_epoch=True)
    self.log('train/acc_epoch', self.train_acc, on_step=False, on_epoch=True)
    self.log('train/f1_epoch', self.train_f1, on_step=False, on_epoch=True)

    self.log("train_loss", loss)
    self.log("train_f1", self.train_f1)
    return {"loss": loss, "predictions": preds, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    preds = torch.argmax(outputs, 1)

    self.val_acc(preds, labels)
    self.val_f1(preds, labels)
    self.log('val/loss_epoch', loss, on_step=False, on_epoch=True)
    self.log('val/acc_epoch', self.val_acc, on_step=False, on_epoch=True)
    self.log('val/f1_epoch', self.val_f1, on_step=False, on_epoch=True)

    self.log("val_loss", loss, prog_bar=True, logger=True)
    self.log("val_f1", self.val_f1, prog_bar=True, logger=True)

    torch.cuda.empty_cache()
    return loss

  # def validation_epoch_end(self, validation_step_outputs):

  # #   # Saving our model weights and biases every epoch.
  # #   # This way, if we overfit, we can just roll back our weights to the saved weights at the best epoch
  # #   dummy_input_dims = torch.zeros(self.hparams['input_dims'], dtype=torch.long, device=self.device)
  # #   dummy_attn_mask = torch.zeros(self.hparams['attention_mask'], dtype=torch.long, device=self.device)
  # #   input_names = [ "input_dims", "attention_mask" ]
  # #   output_names = [ "input_dims", "attention_mask" ]
  # #   dummy_input = [dummy_input_dims, dummy_attn_mask]
  # #   model_filename = f'model_{str(self.global_step).zfill(5)}.onnx'
  # #   torch.onnx.export(self, dummy_input, model_filename,
  # #                     input_names=input_names,
  # #                     output_names=output_names)
  # #   wandb.save(model_filename)

  #   # flatten validation step outputs as a pytorch tensor and turn into a histogram
  #   flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
  #   self.logger.experiment.log(
  #       {'valid/logits': wandb.Histogram(flattened_logits.to('cpu')),
  #         'global_step': self.global_step}
  #   )

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    preds = torch.argmax(outputs, 1)

    self.test_acc(preds, labels)
    self.test_f1(preds, labels)
    self.log('test/loss_epoch', loss, on_step=False, on_epoch=True)
    self.log('test/acc_epoch', self.test_acc, on_step=False, on_epoch=True)
    self.log('test/f1_epoch', self.test_f1, on_step=False, on_epoch=True)

    self.log("test_loss", loss, prog_bar=True, logger=True)
    self.log("test_f1", self.test_f1, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):

    optimizer = AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )


steps_per_epoch=len(fine_tune_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

model = ToxicCommentTagger(
    'finetuned_multilingual_bert_1000ex_512tokens_8bs_pruning.ckpt'
    num_classes=2,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
    )

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
    )

multiplicative=lambda epoch: 1.1 # modifies learning rate after unfreeze
backbone_finetuning = BackboneFinetuning(5, multiplicative) # freezes pretrained model until 5 epochs

model_pruning = ModelPruning(
    pruning_fn='l1_unstructured',
    amount=0.1,
    use_global_unstructured=True,
    )

wandb.login()
wandb_logger = WandbLogger(project="weak-supervision-fsdl-project", entity="weak-supervision-classifier-team")

trainer = pl.Trainer(
    logger=[wandb_logger],
    log_every_n_steps=50,
    checkpoint_callback=checkpoint_callback,
    callbacks=[backbone_finetuning, model_pruning], # , early_stopping_callback, quantization_aware_training]
    max_epochs=N_EPOCHS,
    gpus=1,
    deterministic=True,
    progress_bar_refresh_rate=30,
    precision=16
    )

trainer.fit(model, data_module)

model_ckpt = "finetuned_distill_bert_1000ex_256tokens_8bs_pruning.ckpt"
trainer.save_checkpoint(model_ckpt)
wandb.save('*ckpt*')
wandb.finish()

# Save the model
torch.save(model_ft, './outputs/model.pth')

# Save the labels
with open('./outputs/labels.txt', 'w') as f:
    f.writelines(["%s\n" % item  for item in class_names])

### 3.2 Create and run a PyTorch ScriptRunConfig

In [ ]:
%%writefile conda_dependencies.yml

channels:
  - conda-forge
dependencies:
  - python=3.6
  - pip:
    - azureml-defaults
    - torch
    - pytorch-lightning==1.2.8
    - transformers==4.5.1
    - wandb

In [ ]:
env = Environment.from_conda_specification(name='pytorch-lightning-1.2.8-gpu', file_path='./conda_dependencies.yml')

# specify a GPU base image
env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
)

In [ ]:
args = [
    '--data-folder', toxic _ds.as_named_input('simpsons').as_mount(),
    '--num-epochs', 10
]

project_folder = "./trainingscripts"

config = ScriptRunConfig(
    source_directory = project_folder, 
    script = 'train.py', 
    compute_target=gpu_instance,
    environment = pytorch_env,
    arguments=args,
)

In [ ]:
run = exp.submit(config)

In [ ]:
# Show the PyTorch estimator
RunDetails(run).show()

In [ ]:
# # Load a historic run
# previousRunId = ''
# run = [r for r in exp.get_runs() if r.id == previousRunId][0]
# RunDetails(run).show()

### 3.3 Register the model in Azure ML

In [ ]:
model = run.register_model(model_name='Simpsons-PyTorch',
                           model_path='outputs',
                           model_framework='PyTorch',
                           model_framework_version='1.6',
                           description="Simpsons PyTorch Classifier (From Jupyter Notebook)",
                           tags={'Conference':'Test 42'},
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=2))

print("Model '{}' version {} registered ".format(model.name,model.version))

### 3.4 Download & Test the model

In [ ]:
model.download(exist_ok=True)

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import json
import urllib
from PIL import Image

# Load the model
loaded_model = torch.load(os.path.join('outputs','model.pth'), map_location=lambda storage, loc: storage)
loaded_model.eval()

# Load the labels
with open(os.path.join('outputs','labels.txt'), 'rt') as lf:
    global labels
    labels = [l.strip() for l in lf.readlines()]

    
def scoreImage(image_link):
    # Load the image to predict
    input_image = Image.open(image_link)

    # Pre process
    preprocess = transforms.Compose([
        transforms.Resize(225),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    # Predict the image
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        loaded_model.to('cuda')

    with torch.no_grad():
        output = loaded_model(input_batch)

    index = output.data.cpu().numpy().argmax()
    probability = torch.nn.functional.softmax(output[0], dim=0).data.cpu().numpy().max()

    #Return the result
    return {"label": labels[index], "probability": round(probability*100,2)}



path = r"data/test"
grid = AxesGrid(plt.figure(1, (20,20)), 111, nrows_ncols=(4, 5), axes_pad=0, label_mode="1")

i = 0
for img in os.listdir(path):
    
    #Score the image
    result = scoreImage(os.path.join(path,img))
    
    # Download image
    image = cv2.imread(os.path.join(path,img))
    image = cv2.resize(image, (352, 352))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    cv2.rectangle(image, (0,260),(352,352),(255,255,255), -1)
    cv2.putText(image, "{} - {}%".format(result['label'],result['probability']),(10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.65,(0,0,0),2,cv2.LINE_AA)    
    
    # Show image in grid
    grid[i].imshow(image)
    i = i+1

## 4 Deploy the model

### 4.1 Create a scoring script

In [ ]:
%%writefile 'score.py'
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import json
import urllib
from PIL import Image

from azureml.core.model import Model

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs','model.pth')
    labels_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs','labels.txt')
    
    print('Loading model...', end='')
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.eval()
    
    print('Loading labels...', end='')
    with open(labels_path, 'rt') as lf:
        global labels
        labels = [l.strip() for l in lf.readlines()]
    print(len(labels), 'found. Success!')

    
def run(input_data):
    url = json.loads(input_data)['url']
    urllib.request.urlretrieve(url, filename="tmp.jpg")
    
    input_image = Image.open("tmp.jpg")

    preprocess = transforms.Compose([
        transforms.Resize(225),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    index = output.data.cpu().numpy().argmax()
    probability = torch.nn.functional.softmax(output[0], dim=0).data.cpu().numpy().max()

    result = {"label": labels[index], "probability": round(probability*100,2)}
    os.remove("tmp.jpg")
    return result

### 4.2 Create an environment file

In [ ]:
myenv = Environment(name="weak-supervision-inference")

conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
#conda_dep.add_conda_package("tensorflow")
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("pytorch-lightning==1.2.8")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep



### 4.3 Create an Inference config

In [ ]:
inference_config = InferenceConfig(
    entry_script="score.py", 
    environment=myenv
)

### 4.4 Deploy to ACI

In [ ]:
model = Model(ws, name='Simpsons-PyTorch')
print("Loaded model version:",model.version)

In [ ]:
# Create a deployment config
deploy_config = AciWebservice.deploy_configuration(
                    cpu_cores = model.resource_configuration.cpu, 
                    memory_gb = model.resource_configuration.memory_in_gb,
                    description='Simpson Lego Classifier')

In [ ]:
# Deploy the model to an ACI
aci_service = Model.deploy(ws, 
                name="simpsons-pt-aci", 
                models = [model], 
                inference_config = inference_config, 
                deployment_config = deploy_config, 
                overwrite = True)

aci_service.wait_for_deployment(show_output=True)

In [ ]:
# Connect to previous deployment
aci_service = AciWebservice(ws, "simpsons-pt-aci")

In [ ]:
print("Scoring endpoint:",aci_service.scoring_uri)

### 4.4 Deploy to Azure Kuberneter Service

In [ ]:
aks_target = AksCompute(ws,"Exodus")

deployment_config = AksWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 2)

aks_service = Model.deploy(workspace=ws, 
                       name="simpsons-pytorch-test-6", 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=deployment_config, 
                       deployment_target=aks_target,
                       overwrite=True)

aks_service.wait_for_deployment(show_output = True)

In [ ]:
### Connect to a previous deployed service
aks_service = [r for r in AksWebservice.list(ws) if r.name == 'simpsons-pytorch'][0]

In [ ]:
print(aks_service.scoring_uri)
aks_service.get_keys()

### 4.4 Test the API

In [ ]:
%%writefile 'test-images-urls.txt'
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Krusty.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Bart.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Flanders.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Homer.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Lisa.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/marge.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Milhouse.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/MrBurns.jpg
https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Wiggum.jpg

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from PIL import Image as ImagePil
import requests
from io import BytesIO
import cv2
import urllib
import cv2
import numpy as np
import json

F = plt.figure(1, (20,20))
grid = AxesGrid(F, 111, nrows_ncols=(2, 3), axes_pad=0, label_mode="1")

with open('test-images-urls.txt', 'rt') as lf:
    global testimages
    testimages = [l.strip() for l in lf.readlines()]
    
def url_to_image(url):
    with urllib.request.urlopen(url) as url:
        s = url.read()
    image = np.asarray(bytearray(s), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (352, 352))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


i = 0
for img_name in testimages[0:6]:
    
    # Predict Url ACI
    #result = aci_service.run(input_data=json.dumps({ "url": img_name}))
    
    # Predict Url AKS
    result = aks_service.run(input_data=json.dumps({ "url": img_name}))

    # Download image
    img = url_to_image(img_name)
 
    # Draw result on image
    cv2.rectangle(img, (0,260),(352,352),(255,255,255), -1)
    cv2.putText(img, "{} - {}%".format(result['label'],result['probability']),(10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.65,(0,0,0),2,cv2.LINE_AA)

    # Show image in grid
    grid[i].imshow(img)
    i = i+1

In [ ]:
with open('test-images-urls.txt', 'rt') as lf:
    global testimages
    testimages = [l.strip() for l in lf.readlines()]

print(testimages)